<a href="https://colab.research.google.com/github/sameermohan-git/chatgpt/blob/main/ChatGPT_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install openai

import os
import pandas as pd
import openai
import subprocess

In [ ]:
openai.api_key = 'sk-XGnFJqfHy24ZTlJDThcE0seK8'#os.getenv("OPENAI_API_KEY")

l_age = ['18', '20']
l_gender = ['man']
l_power = ['invisibility'] 

f_prompt = "Imagine a complete and detailed description of a {age}-year-old {gender} fictional character who has the superpower of {power}. Write out the entire description in a maximum of 100 words in great detail:"
f_sub_prompt = "{age}, {gender}, {power}"

df = pd.DataFrame()
for age in l_age:
 for gender in l_gender:
  for power in l_power:
   for i in range(3): ## 3 times each
    prompt = f_prompt.format(age=age, gender=gender, power=power)
    sub_prompt = f_sub_prompt.format(age=age, gender=gender, power=power)
    print(sub_prompt)

    response = openai.Completion.create(
     model="text-davinci-003",
     prompt=prompt,
     temperature=1,
     max_tokens=500,
     top_p=1,
     frequency_penalty=0,
     presence_penalty=0
    )
    
    finish_reason = response['choices'][0]['finish_reason']
    response_txt = response['choices'][0]['text']
    
    new_row = {
      'age':age, 
      'gender':gender, 
      'power':power, 
      'prompt':prompt, 
      'sub_prompt':sub_prompt, 
      'response_txt':response_txt, 
      'finish_reason':finish_reason}
    new_row = pd.DataFrame([new_row])
    df = pd.concat([df, new_row], axis=0, ignore_index=True)

df.to_csv("out_openai_completion.csv")

In [ ]:
import subprocess

df = pd.read_csv("PycharmProjects/chatgpt/out_openai_completion.csv")

prepared_data = df.loc[:,['sub_prompt','response_txt']]
prepared_data.rename(columns={'sub_prompt':'prompt', 'response_txt':'completion'}, inplace=True)
prepared_data.to_csv('PycharmProjects/chatgpt/prepared_data.csv',index=False)


## prepared_data.csv --> prepared_data_prepared.json
subprocess.run('openai tools fine_tunes.prepare_data --file PycharmProjects/chatgpt/prepared_data.csv --quiet'.split())

## Start fine-tuning
subprocess.run('openai --api-key sk-XGn8ln8ePYlNBGDHglbkFJqfHy24ZTlJDThcE0seK8 api fine_tunes.create --training_file PycharmProjects/chatgpt/prepared_data_prepared.jsonl --model curie --suffix "SameerMohan"'.split())
